# Проверка на простоту

*Число $m > 1$ является простым числом тогда и только тогда, когда
$$a^{m-1} \equiv 1 \mod m$$
для всех $a \not\equiv 0 \mod m$.*

Это следует с одной стороны из малой теоремы Ферма.
Если же $m$ составное, то есть $1 < a < m$, делящее $m$.
Предположим, что и в этом случае $a^{m-1} \equiv 1 \mod m$.
Тогда $a \mid m$ и $m \mid a^{m-1} - 1$.
Следовательно, $a \mid a^{m-1} - 1$, что значит $a k = a^{m-1} - 1$ для некоторого $k$ или $a^{m-1} - a k = 1$, т.е. $a \mid 1$.
Противоречие.

С помощью этого факта можно быстро доказать, что число составное.
Скажем, если $2^{m-1} \not\equiv 1 \mod m$, то число $m$ составное.

Но если для нескольких разных значений $a$, допустим, для всех $a$ взаимно простых с $m$, $\gcd(a,m)=1$, увидеть это не удалось, то возможно число $m$ простое?

Не обязательно (хотя и такой вероятностный тест применяют).
Существуют числа, которые проваливают такой "тест": например, $2^{1729-1} \equiv 1 \mod 1729$, но $1729 = 7 \cdot 13 \cdot 19$.

## Возведение в степень по модулю

Сначала научимся выполнять основную операцию для такой проверки.

**Задача.**
Напишите функцию `powmod(a,b,m)`, которая для целых $a,b$ и $m$ будет возвращать число $a^b \mod m$.

*Конечно, использовать встроенную функцию возведения в степень по модулю нельзя.*

Самый очевидный алгоритм, конечно,
```python
def powmod_easy(a, b, m):
    s = 1
    for i in xrange(b):
        s = (s * a) % m
    return s
```
Этот алгоритм выполняется за $O(b)$ операций умножения и взятия остатка.
Если $b$ велико, а на простоту проверяются обычно очень большие числа, то заниматься этим вычислением можно вечно.

Рассмотрим алгоритм сложность которого $O(\log b)$: понадобится всего $\log_2 b$ умножений по модулю.

Будем использовать двоичное представлени $b = (b_s b_{s-1} \ldots b_1 b_0)_2$ и искать 
$$(a^{2^0})^{b_0} \cdot (a^{2^1})^{b_1} \cdot (a^{2^2})^{b_2} \cdot \ldots \cdot (a^{2^s})^{b_s} \mod m = a^{b_0} \cdot (a^{2})^{b_1} \cdot ((a^{2})^2)^{b_2} \cdot \ldots \cdot ((a^{2^{s-1}})^{2})^{b_s} \mod m$$
в цикле.
На шаге $i = 0, 1, \ldots, s$ вычисляем $(a^{2^{i-1}})^2$ и просматриваем бит $b_i$.

**Задача.**
Проведите пару испытаний написанной функции `powmod`.
Cравните время выполнения с наивным примером `powmod_easy` (и при этом не повесьте компьютер), а также со встроенной `pow` или `power_mod`:
```python
timeit('powermod(2, 10^6 - 1, 10^6)')
```


## Числа Кармайкла

Числом Кармайкла (Carmichael) называется всякое составное число $m$, такое, что для всех $a$ взаимно простых с $m$ выполняется $a^{m-1} \equiv 1 \mod m$.

**Задача.**
Найдите самое маленькое число Кармайкла

а затем и

**Задача.**
Найдите все составные числа $m$ до некоторого $M$, такие, что для всех $a < m$ и $\gcd(a,m)=1$ выполнено $$a^{m-1} \equiv 1 \mod m$$

Проверять целое число `n` на простоту можно методом `n.is_prime()` или `is_prime(n)`, а для поиска наибольшего общего делителя двух чисел `x` и `y` есть `x.gcd(y)` или функция `gcd(x,y)`.

## Тест Миллера-Рабина

Этот тест используется, например, для проверки сгенерированных чисел на простоту при создании ключей в действующих программных реализациях шифрования с открытым ключом.

**Алгоритм (MR):**

1. Выделить в $n-1$ максимальную степень 2: найти такие $k$ и $m$, что $n-1 = 2^k m$ и число $m$ нечетное.
2. Выбрать произвольное целое число $a$ с условием $1 < a < n$.
3. Вычислить $b = a^m \bmod n$.
   
   Если $b \equiv \pm 1 \mod n$ выдать `True`.
4. Если $b^{2^r} \equiv -1 \mod n$ для некоторого $r$, $1 \leq r \leq k-1$, выдать `True`.

   Если $b^{2^r} \equiv 1 \mod n$ для некоторого $r$, $1 \leq r \leq k-1$, выдать `False`.
   
   Иначе выдать `False`.

Если Алгоритм выдает `True` то число считается вероятно простым, а если `False`, то число составное.
Чтобы уменьшить вероятность ошибки можно повторить алгоритм, начиная с шага 2.

Основой этого алгоритма является свойство всякого простого числа $p$:

*По простому модулю $p$ существуют только тривиальные корни из единицы, т.е. сравнение $x^2 \equiv 1\mod p$ имеет только два решения, $1$ и $-1$.*

Это обусловлено тем, что многочлен над полем не может иметь больше корней, чем его степень.

Действительно, можно убедиться в том, что для всякого простого Алгоритм MR вернет `True`.

Предположим, что число $n$ простое, но алгоритм вернул `False`.
Значит на шаге 3 получаем $a^m \not\equiv \pm 1 \mod n$, на шаге 4 получили $a^{2^r m} \not\equiv -1 \mod n$ для всех $1 \leq r \leq k-1$.

Согласно [критерию Эйлера](https://ru.wikipedia.org/wiki/Критерий_Эйлера), для простого $n>2$ имеет место $a^{\frac{n-1}{2}} \equiv \pm 1 \mod n$.
То есть, по предположению, $a^{\frac{n-1}{2}} = a^{2^{k-1}m} \equiv 1 \mod n$.
Так как нетривиальных корней по $n$ нет, то $a^{2^{k-2}m} \equiv \pm 1 \mod n$, а по предположению $a^{2^{k-2}m} \equiv 1 \mod n$. И так далее, вплоть до $a^{m} \equiv 1 \mod n$, что противоречит начальному предположению.

*Рабин (Rabin Michael O. Probabilistic algorithm for testing primality, 1980) показал: вероятность того, что составное нечетное число будет названо простым после $k$-ой попытки, не превышает $\frac{1}{4^k}$.*

**Задача.** Напишите функцию `mr(n)`, реализующую тест Миллера-Рабина для числа $n$.

**Задача.** Напишите функцию `mr(n,k)`, реализующую тест Миллера-Рабина для числа $n$ с числом раундов $k$.

##  Тест Люка

*Если для некоторого числа $a$, $1 < a < n$, такого, что $a^{n-1} \equiv 1 \mod n$, выполняются сравнения $a^{\frac{n-1}{q}} \not\equiv 1 \mod p$ по всем делителям $q$ числа $n-1$, то $n$ простое.
Если такого числа $a$ не существует, то $n$ составное.*

В самом деле, в этом случае порядок элемента $a$ равен $n-1$ и его степени пробегают все ненулевые вычеты по модулю $n$.
Поэтому они образуют группу, а значит каждый ее элемент обратим и взаимно прост с $n$.

# Сердце системы RSA

$N = p \cdot q$

$e \cdot d \equiv 1 \mod \phi(N)$

`pk = (N, e)` и `sk = (N, d)`

$E(m) = m^e \mod N$

$D(m) = m^d \mod N$

$D(E(m)) = (m^e)^d \mod N = m^{e d} \mod N$, но $e d - 1 = \phi(N) \cdot k$, поэтому $m^{e d} \mod N = m^{\phi(N) k + 1} \mod N = (m^{\phi(N)})^k \cdot m \mod N = m \mod N$.
Получается $D(E(m)) = m$.

Обратный по модулю $\phi(N)$ можно найти только для числа $e$ взаимнопростого с модулем, значит
$(e, \phi(N)) = 1$. Тогда существуют $x, y$, что $1 = e \cdot x + \phi(N) \cdot y$ или $ex \equiv 1 \mod \phi(N)$.
Таким образом, для нахождения обратного к $e$ сойдет расширенный алгоритм Евклида, примененный к $e$ и $\phi(N)$.

Для расширенного алгоритма Евклида есть функция `xgcd` или специальная `inverse_mod`.